In [1]:
import pandas as pd
import json
import numpy as np
import time 
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from pymongo import MongoClient

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We need to download 6 different categories, we want to create different collections

In [4]:
dataset1_review = load_dataset("McAuley-Lab/Amazon-Reviews-2023","raw_review_All_Beauty", split="full", trust_remote_code=True)
dataset1_meta = load_dataset("McAuley-Lab/Amazon-Reviews-2023","raw_meta_All_Beauty", split="full", trust_remote_code=True)

dataset2_review = load_dataset("McAuley-Lab/Amazon-Reviews-2023","raw_review_Cell_Phones_and_Accessories", split="full", trust_remote_code=True)
dataset2_meta = load_dataset("McAuley-Lab/Amazon-Reviews-2023","raw_meta_Cell_Phones_and_Accessories", split="full", trust_remote_code=True)

dataset3_review = load_dataset("McAuley-Lab/Amazon-Reviews-2023","raw_review_Video_Games", split="full", trust_remote_code=True)
dataset3_meta = load_dataset("McAuley-Lab/Amazon-Reviews-2023","raw_meta_Video_Games", split="full", trust_remote_code=True)


datasets_meta = [dataset1_meta,dataset2_meta,dataset3_meta]
datasets_reviews = [dataset1_review,dataset2_review,dataset3_review]

In [2]:
class ConnectDB:
    def __init__(self) -> None:
        self.client = client = MongoClient("mongodb://localhost:27017")
        self.db = db = client.massiveData
    
    def create_collection(self,name):
        collection = self.db[name]
        return None

    def get_collection(self,name):
        collection = self.db[name]
        return collection

# products_df = pd.DataFrame()

class InfoLoader:
    def __init__(self):
        pass

    def create_review(self,dataset, date= '2023-01-01'):
        """
        Prepare a dataset to be added to a collection.
        Input:
            - Array of raw metadata of reviews.
            - Date since we want the first review.
        Output:
            - Reviews transformed to be added to a collection filtered for a timestamp.
        """
        reviews = []

        i = 0
        for review in dataset:


            # Calculate the timestamp from 01/01/2022 
            ts_2022 = pd.Timestamp(date).timestamp() * 1000


            if review['verified_purchase'] is True and review['timestamp'] > ts_2022:
            
                aux_review = {}
                aux_review['rating'] = review['rating']

                #Que tengan un titulo similar 
                aux_review['title'] = review['title']

                #Analisis de sentimientos o reviews parecidas
                aux_review['text'] = review['text']

                #Producto asociado recomendar productos no comprados
                aux_review['id_product'] = review['parent_asin']

                #Producto asociado
                aux_review['id_user'] = review['user_id']

                #Compra verificada
                aux_review['verified_purchase'] = review['verified_purchase']

                aux_review['helpful_vote'] = review['helpful_vote']


                aux_review['url'] = 'https://amazon.com.mx/' + review['parent_asin']

                reviews.append(aux_review)

        return reviews

    def create_metadata(self,dataset):
        """
        Prepare a dataset to be added to a collection
        input: 
            Array of raw metadata of products
        output:
            Products transformed to be added to a collection
        """
        products = []
        for product in dataset:

            aux_product = {}

            aux_product['id'] = product['parent_asin']

            #Que tengan un titulo similar 
            aux_product['title'] = product['title']


            #Que tengan una categoria similar 
            aux_cat = ''
            #Add any categories that the product has, limited by 3
            for i,cat in enumerate(product['categories']):
                aux_cat += cat + ','
            aux_product[f"categorie"] = aux_cat
            

            aux_product['price'] = product['price'] or -1
            
            # Que tengan descripcion similar
            aux_product["description"] = product['description'] or np.nan

            aux_product["features"] = product['features'] or np.nan

            if len(product['images']['large']) > 0:
                aux_product["img"] = product['images']['large'][0] or np.nan
            else:
                aux_product["img"] = ''
                
            aux_product['avg_rating'] = product['average_rating'] or np.nan

            aux_product['url'] = 'https://amazon.com.mx/' + product['parent_asin']

            products.append(aux_product)


        return products



## Adding the Reviews and Metadata to the database

1. Create 2 collections 
    1. Reviews
    2. Products
2. Transform the original datasets for both meta and reviews for each category
3. Load collections and add the transformed datasets to their respective collection

In [3]:
db = ConnectDB()
infl = InfoLoader()


# Step 1: Create 2 collections
db.create_collection('reviews')
db.create_collection('products')

The time it took to finish this ETL was 215 minutes.

In [23]:

# Step 2: Transform the original datasets
datasets_meta_transformed = []
datasets_reviews_transformed = []
for meta,review in zip(datasets_meta,datasets_reviews):
    datasets_meta_transformed.append(infl.create_metadata(meta))
    datasets_reviews_transformed.append(infl.create_review(review))
    

to_meta =  [item for sublist in datasets_meta_transformed for item in sublist]
to_reviews = [item for sublist in datasets_reviews_transformed for item in sublist]

products_df = pd.DataFrame(to_meta)
products_df.to_csv('./datasets_meta_transformed.csv',index=False)

reviews_df = pd.DataFrame(to_reviews)
reviews_df.to_csv('./datasets_reviews_transformed.csv',index=False)

#### Load to the database

In [24]:
df_reviews = pd.read_csv('./datasets_reviews_transformed.csv')

reviews_exported = df_reviews.to_dict(orient='records')

reviews_collection = db.get_collection('reviews')

reviews_collection.insert_many(reviews_exported)

In [ ]:
df_meta = pd.read_csv('./datasets_meta_transformed.csv')

meta_exported = df_meta.to_dict(orient='records')

products_collection = db.get_collection('products')

products_collection.insert_many(meta_exported)

In [4]:
products_collection = db.get_collection('products')
products_collection.update_many({}, {"$rename": {"id": "id_product"}})

UpdateResult({'n': 1538349, 'nModified': 1538349, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)